In [ ]:
# section 1 load all the necessary modules and packages
import numpy        as np
import pandas       as pd
import glob
import sys
import os

### hello guys!
### good afternoon!

class Energy:

    def __init__(self):
        
        # atm forcing
        self.atm_T       = 10 # [C]
        self.VegH        = 10 # [m]
        self.MoistH      = 10 # [m]
        self.WindH       = 10 #[m]
        self.atm_wind    = 0.2  # [m s-1]
        #
        self.rs_min      = 200  # [s-1 m]
        self.rs_max      = 5000  # [s-1 m]
        #
        self.rs_rp       = 100 # [W m-2] or [j m-2 s-1] for trees minimum radiation for photosynthsis
        self.rs_rp       =  30 # [W m-2] or [j m-2 s-1] for crops minimum radiation for photosynsis
        
        #
        self.large_value = 10**10
        
        # parameters
        self.wilting = 0.3 # [-]
        self.no_stress_moisture = 0.7 # [-]
        
        # constant and hard coded parameters
        self.gamma = 66          # [Pa/k] or [kg m-1 s2 K-1]
        self.lv    = 2453*10**6  # [J m−3]
        self.ro_air= 1           # [kg m-3]
        self.sh_air= 10          # [J kg-1 K-1]
        
        
        
        self.emissivity = 1
        # 
        
#         self.atm_SWR
#         self.atm_LWR
#         self.atm_VP
#         self.atm_wind
#         self.atm_XXX
#         # paraemter that
        self.par_LAI = 2
#         self.par_stm_r

        self.atm_Q = 0.01
        
        
        self.rs_moisture_f = 0
        
    def fun_es(self):
        # IN: T, Temperature (deg C)
        # OUT: es, Saturation vapour pressure (Pa)
        self.es = 610.8*np.exp((17.27*self.atm_T)/(self.atm_T+237.3))
    
    def fun_es_d(self): # slope of saturation presure
        # IN: Temperature (deg C)
        # OUT: Slope of the saturation vapour pressure curve (Pa/deg C)
        self.es_d = (4098*self.es)/((self.atm_T+237.3)**2)

    def fun_ra(self):
        # IN: heights of measurements and vegetation (m)
        # OUT: aerodynmamic resistance (s/m)
        zom = 0.123*self.VegH #roughness length of momentum transfer
        zoh = 0.10*zom #roughness length of heat and vapor transfer
        d   = 0.67*self.VegH #zero displacment height
        self.ra = (np.log((self.MoistH-d)/zom)*np.log((self.WindH-d)/zoh))/(self.atm_wind*0.40**2)
        
    def fun_stomata_SWR(self):
        # IN: heights of measurements and vegetation (m)
        # OUT: aerodynmamic resistance (s/m)
        self.cs_SWR_f = 1/((self.rs_min/self.rs_max+self.atm_SWR/self.rs_rp)/(1+self.atm_SWR/self.rs_rp))
        
    def fun_stomata_VPD(self):
        # IN: vapour pressure and saturate vapour pressure [Pa]
        # out: vapour pressure deficit conductant [m s-1]
        self.cs_VPD_f = 1/ (1-(self.es-self.e)/4)
        
    def fun_stomata_temp(self):
        # IN: stmospehric temprature
        # OUT: temprature conductant
        self.cs_temp_f = 1/(0.08*self.atm_T-0.0016*self.atm_T**2)
        if self.cs_temp_f > self.large_value:
            self.cs_temp_f = self.large_value
        if self.cs_temp_f < 0:
            self.cs_temp_f = self.large_value
        
    def fun_stomata_moisture(self):
        # IN: soil mositure [-]
        # OUT: soil mositure conductance
        if self.no_stress_moisture < self.wilting:
            sys.exit('no stress moisture cannot be less than wilting point mositure')
        if self.moisture < self.wilting:
            self.cs_moisture_f = self.large_value
        if (self.wilting <= self.moisture) and (self.moisture <= self.no_stress_moisture):
            self.cs_moisture_f = (self.no_stress_moisture-self.wilting)/(self.moisture-self.wilting)
        if self.no_stress_moisture < self.moisture:
            self.cs_moisture_f = 1
        if self.cs_moisture_f > self.large_value:
            self.cs_moisture_f = self.large_value
    
    def fun_rs(self):
        # IN: temprautre, mositure, VPD ad SWR conductace
        # OUT: stomata resitance
        self.rs = self.rsmin * (1/self.cs_SWR_f)*(1/self.cs_VPD_f)*(1/self.fun_cs_temp)*(1/self.cs_moisture_f)
        
    def fun_rs_scale(self):
        # IN: temprautre, mositure, VPD ad SWR conductace
        # OUT: stomata resitance
        self.rs_scale = self.rs / (0.5*self.LAI)
        
    def fun_Penman_Monteith(self):
        # IN: net radiation, slope of vapour pressure, air density,
        # IN: vapour pressure, saturated vapour pressure, aerodynamic resitance,
        self.E = (self.es_d * self.net_rad + self.ro_air *self.sh_air * (self.es - self.e) * (1/self.ra))/\
        (seld.es_d + self.gamma * (1 + (self.rs+self.rarc)/self.ra))
        
    def fun_net_rad(self):
        # IN: short wave radiation, long wave radiation, emissivity
        # OUT: netradiation, ground flux
        self.net_rad = (1-self.albedo)*self.atm_SWR + self.emissivity* self.atm_LWR +\
        self.emissivity * (5.67*10**-8) * self.atm_T**4
        self.ground = 0.1 * self.net_rad
        
    def fun_Q_e (self):
        # IN: specific humidity and atmospheric pressure
        # OUT: vapour pressure
        self.e = self.atm_Q * self.atm_P / (0.378 * self.atm_Q + 0.622)

        

In [ ]:
# initialize
Eng = Energy()

Eng.atm_T = 100.0 # C

Eng.fun_es()
Eng.fun_es_d()
print(Eng.es)
print(Eng.es_d)
E1 = Eng.es
print('----')

Eng.atm_T = 102.0 # C

Eng.fun_es()
Eng.fun_es_d()
print(Eng.es)
E2 = Eng.es
print(Eng.es_d)
print('----')

D = (E2 - E1)/2
print(D)

In [ ]:
import matplotlib.pyplot as plt
# initialize
Eng = Energy()

#print(Eng.wilting)
#print(Eng.no_stress_moisture)

# parameters
Eng.wilting = 0.3 # [-]
Eng.no_stress_moisture = 0.35 # [-]

saturation_arr = np.linspace(0, 1, num=101)
resistance = np.zeros([101,])

for i in np.arange(len(saturation_arr)):
    # print(saturation_arr[i])
    Eng.moisture = saturation_arr[i]
    
    Eng.fun_stomata_moisture()
    resistance [i] = Eng.cs_moisture_f
    
    
    #print(i,saturation_arr[i], Eng.moisture, resistance [i])
    
plt.plot(saturation_arr, 1/resistance)
plt.grid()

